<a href="https://colab.research.google.com/github/abhayjeet5465/FIN_ADVISOR/blob/main/finAdvisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================================
# STEP 1 — install all dependencies
# ===============================================================
!pip install -q -U unsloth transformers accelerate bitsandbytes datasets peft

# basic imports
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from unsloth import FastLanguageModel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
!ls /content


indian_finance_dataset_200.csv	sample_data  unsloth_compiled_cache


In [ ]:
# ===============================================================
# STEP 2 — load datasets
# ===============================================================

# base dataset from Hugging Face
base_ds = load_dataset("gbharti/finance-alpaca", split="train[:5000]")   # take 5k examples for demo

# your new CSV
local_path = "/content/indian_finance_dataset_200.csv"  # adjust if in /drive
my_ds = load_dataset("csv", data_files=local_path)["train"]

print("base_ds:", len(base_ds))
print("my_ds:", len(my_ds))
print(base_ds[0])
print(my_ds[0])

# unify column names -> 'instruction', 'response'
def format_gbharti(ex):
    return {
        "instruction": ex.get("instruction", ""),
        "response": ex.get("response", ex.get("output", "")),
    }

base_ds = base_ds.map(format_gbharti, remove_columns=base_ds.column_names)
my_ds = my_ds.rename_columns({"instruction": "instruction", "response": "response"})

# merge + shuffle
merged_ds = concatenate_datasets([base_ds, my_ds]).shuffle(seed=42)
print("✅ merged dataset size:", len(merged_ds))


Generating train split: 0 examples [00:00, ? examples/s]

base_ds: 5000
my_ds: 200
{'instruction': 'For a car, what scams can be plotted with 0% financing vs rebate?', 'input': '', 'output': "The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 36,48,60 or 

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

✅ merged dataset size: 5200


In [ ]:
for i in range(3):
    print(f"INSTRUCTION: {merged_ds[i]['instruction']}\nRESPONSE: {merged_ds[i]['response']}\n{'-'*80}")


INSTRUCTION: How do I enter Canadian tax info from US form 1042-S and record captial gains from cashing in stock options?
RESPONSE: I can't give you a specific answer because I'm not a tax accountant, so you should seek advice from a tax professional with experience relevant to your situation. This could be a complicated situation. That being said, one place you could start is the Canada Revenue Agency's statement on investment income, which contains this paragraph: Interest, foreign interest and dividend income, foreign income,   foreign non-business income, and certain other income are all amounts   you report on your return. They are usually shown on the following   slips: T5, T3, T5013, T5013A To avoid double taxation, Canada and the US almost certainly have a foreign tax treaty that ensures you are only taxed in your country of residence. I'm assuming you're a resident of Canada. Also, this page states that: If you received foreign interest or dividend income, you have to   report

In [ ]:
# ===============================================================
# STEP 4 — load model + tokenizer
# ===============================================================
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-3B-Instruct",
    load_in_4bit = True,          # fits comfortably on Colab T4
    max_seq_length = 2048,
)

print("✅ Model and tokenizer loaded successfully!")


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ Model and tokenizer loaded successfully!


In [ ]:
# ===============================================================
# STEP 5 — tokenize dataset
# ===============================================================
def preprocess_function(examples):
    texts = [
        f"### Instruction:\n{ins}\n\n### Response:\n{res}"
        for ins, res in zip(examples["instruction"], examples["response"])
    ]
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

tokenized_ds = merged_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=merged_ds.column_names
)

print("✅ Tokenization complete — ready for training.")
print(tokenized_ds)


Map:   0%|          | 0/5200 [00:00<?, ? examples/s]

✅ Tokenization complete — ready for training.
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5200
})


In [ ]:
# ===============================================================
# STEP 7 (revised): add LoRA adapters to quantized model
# ===============================================================
from unsloth import FastLanguageModel

# if not already loaded:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "meta-llama/Llama-3.2-3B-Instruct",
#     load_in_4bit = True,
#     max_seq_length = 2048,
# )

model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",   # attention layers
        "gate_proj", "up_proj", "down_proj"       # MLP layers
    ],
    bias = "none",
)

print("✅ LoRA adapters attached — now trainable even in 4-bit mode.")


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.2 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ LoRA adapters attached — now trainable even in 4-bit mode.


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="finetuned_finance_llama3_3b_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    num_train_epochs=1.0,
    fp16=True,
    optim="paged_adamw_8bit",
    logging_steps=25,
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator,
)

trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,200 | Num Epochs = 1 | Total steps = 1,300
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 12,156,928 of 3,224,906,752 (0.38% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,6.560300
50,6.765500
75,6.603900
100,6.644100
125,6.613700
150,6.663200
175,6.592700
200,6.744400
225,6.836200
250,6.664200


TrainOutput(global_step=1300, training_loss=6.65900630070613, metrics={'train_runtime': 6393.077, 'train_samples_per_second': 0.813, 'train_steps_per_second': 0.203, 'total_flos': 9.0443995938816e+16, 'train_loss': 6.65900630070613, 'epoch': 1.0})

In [ ]:
# ===============================================================
# STEP 9A — save adapters + tokenizer
# ===============================================================
save_path = "finetuned_finance_llama3_3b_lora"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("✅ LoRA adapters + tokenizer saved at", save_path)


✅ LoRA adapters + tokenizer saved at finetuned_finance_llama3_3b_lora


In [ ]:
# ✅ New Unsloth syntax
merged_model = model.merge_and_unload()
merged_model.save_pretrained("finetuned_finance_llama3_3b_full")
tokenizer.save_pretrained("finetuned_finance_llama3_3b_full")

print("✅ Merged full model saved at finetuned_finance_llama3_3b_full")

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


✅ Merged full model saved at finetuned_finance_llama3_3b_full


In [ ]:
# ===============================================================
# STEP 9C — upload to Hugging Face
# ===============================================================
from huggingface_hub import login
login()  # paste your HF token from https://huggingface.co/settings/tokens

# choose what to push:
#  • just adapters
#  • or full merged model (if Step 9B ran)
repo_name = "Abhayjeet/finetuned-finance-llama3-3B-LORA"

model.push_to_hub(repo_name, private=True)
tokenizer.push_to_hub(repo_name, private=True)
print(" Uploaded to Hugging Face:", repo_name)


In [ ]:
!zip -r finetuned-finance-llama3-3b-lora.zip finetuned_finance_llama3_3b_lora
from google.colab import files
files.download("finetuned-finance-llama3-3b-lora.zip")



  adding: finetuned_finance_llama3_3b_lora/ (stored 0%)
  adding: finetuned_finance_llama3_3b_lora/chat_template.jinja (deflated 71%)
  adding: finetuned_finance_llama3_3b_lora/README.md (deflated 65%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/ (stored 0%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/chat_template.jinja (deflated 71%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/README.md (deflated 65%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/rng_state.pth (deflated 26%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/trainer_state.json (deflated 87%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/training_args.bin (deflated 53%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/adapter_config.json (deflated 56%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/scaler.pt (deflated 64%)
  adding: finetuned_finance_llama3_3b_lora/checkpoint-1300/special_tokens_map.json (deflated 71%)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Abhayjeet/finetuned-finance-llama3-3B-LORA",
    load_in_4bit = True,
    max_seq_length = 2048,
)

prompt = """### Instruction:
Suggest an Indian stock portfolio for ₹2 lakh for a medium-risk investor in 2025-26.

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

### Instruction:
Suggest an Indian stock portfolio for ₹2 lakh for a medium-risk investor in 2025-26.

### Response:
As a medium-risk investor, I would suggest a diversified portfolio with a mix of large-cap, mid-cap, and small-cap stocks. Here's a suggested portfolio for ₹2 lakh:

**Large-Cap Stocks (40%): ₹80,000**

1. **HDFC Life Insurance Company Limited (HDFC Life)**: ₹20,000
2. **Reliance Life Insurance Company Limited (RLI)**: ₹15,000
3. **Birlasoft Limited (Birlasoft)**: ₹15,000
4. **Larsen & Toubro Limited (Larsen & Toubro)**: ₹10,000
5. **Tata Consultancy Services Limited (TCS)**: ₹10,000

**Mid-Cap Stocks (30%): ₹60,000**

1. **Airtel Limited (Airtel)**: ₹15,000
2. **Sun Pharma Limited (Sun Pharma)**: ₹10,000
3. **Dalmia Bharat Limited (Dalmia Bharat)**: ₹10,000
4. **Hindustan Copper Limited (Hindustan Copper)**: ₹5,000
5. **Bajaj Finance Limited (Bajaj Finance)**: ₹5,000

**Small-Cap Stocks (30%): ₹60,000**

1. **GMR Infrastructure Limited (GMR Infrastructure)**: ₹15,000


In [ ]:
prompt = """### Instruction: ACT AS A PERSONALIZED INVESTMENT ADVISOR AND BASED ON YOUR TRAINED DATA
i want to invest rupeess 50,000 in indian stock market where should i invest?

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction: ACT AS A PERSONALIZED INVESTMENT ADVISOR AND BASED ON YOUR TRAINED DATA 
i want to invest rupeess 50,000 in indian stock market where should i invest?

### Response:
As a personalized investment advisor, I'll provide you with a tailored recommendation based on your risk tolerance and investment goals.

**Investment Objective:**
Before we begin, please let me know what are your investment objectives?

1. **Income Generation**: Are you looking for regular income through dividends or interest?
2. **Capital Appreciation**: Are you looking to grow your capital over time?
3. **Risk Tolerance**: How comfortable are you with market volatility?

Assuming you're a moderately risk-tolerant investor seeking a balanced approach, I'll recommend a diversified portfolio of Indian stocks.

**Portfolio Allocation:**

1. **Large-Cap Stocks**: 40% ( ₹20,000)
	* Invest in established companies like:
		+ Reliance Industries (RIL)
		+ HDFC Life
		+ Bharti Airtel (Airtel)
		+ Tata Consultancy